# Exercicio Extra - Markov



Você está o dataset **bike.csv**. Ele contém informações de bicicletas de viajam entre vários terminais de uma cidade.

1. Crie a matriz de transição que representa a probabilidade de uma bicicleta sair da estação i, para a estação j.

2. Calcule o tempo médio de retorno de cada estação.

OBS: (start station id) e (end estation id) identificam as estações de inicio e fim da corrida de cada exemplo.



In [1]:
!pip install --upgrade gdown
!gdown 1T41RVee_AaQC0k_52-I14tuCjpkNz0i4

Downloading...
From: https://drive.google.com/uc?id=1T41RVee_AaQC0k_52-I14tuCjpkNz0i4
To: /content/bike.csv
100% 101M/101M [00:00<00:00, 113MB/s] 


In [3]:
import pandas as pd

In [28]:
df = pd.read_csv('bike.csv')

In [29]:
df=df.dropna().reset_index(drop=True)

In [30]:
df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,695,2013-06-01 00:00:01,2013-06-01 00:11:36,444,Broadway & W 24 St,40.742354,-73.989151,434.0,9 Ave & W 18 St,40.743174,-74.003664,19678,Subscriber,1983.0,1
1,693,2013-06-01 00:00:08,2013-06-01 00:11:41,444,Broadway & W 24 St,40.742354,-73.989151,434.0,9 Ave & W 18 St,40.743174,-74.003664,16649,Subscriber,1984.0,1
2,123,2013-06-01 00:01:04,2013-06-01 00:03:07,475,E 15 St & Irving Pl,40.735243,-73.987586,262.0,Washington Park,40.691782,-73.973730,16352,Subscriber,1960.0,1
3,1521,2013-06-01 00:01:22,2013-06-01 00:26:43,2008,Little West St & 1 Pl,40.705693,-74.016777,310.0,State St & Smith St,40.689269,-73.989129,15567,Subscriber,1983.0,1
4,2057,2013-06-01 00:02:33,2013-06-01 00:36:50,285,Broadway & E 14 St,40.734546,-73.990741,532.0,S 5 Pl & S 5 St,40.710451,-73.960876,15693,Subscriber,1991.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326836,436,2013-06-30 23:58:09,2013-07-01 00:05:25,161,LaGuardia Pl & W 3 St,40.729170,-73.998102,403.0,E 2 St & 2 Ave,40.725029,-73.990697,19454,Subscriber,1982.0,1
326837,185,2013-06-30 23:57:52,2013-07-01 00:00:57,323,Lawrence St & Willoughby St,40.692362,-73.986317,293.0,Lafayette St & E 8 St,40.730207,-73.991026,16746,Subscriber,1982.0,1
326838,595,2013-06-30 23:58:39,2013-07-01 00:08:34,360,William St & Pine St,40.707179,-74.008873,329.0,Greenwich St & North Moore St,40.720152,-74.010365,19290,Subscriber,1985.0,1
326839,925,2013-06-30 23:59:27,2013-07-01 00:14:52,509,9 Ave & W 22 St,40.745497,-74.001971,462.0,W 22 St & 10 Ave,40.746920,-74.004519,15250,Subscriber,1964.0,1


In [31]:
df['start station id'].nunique()

337

In [32]:
df['end station id'].nunique()

337

In [33]:
len(set(list(df['start station id'].unique())+list(df['end station id'].unique())))

337

In [34]:
df = df.dropna(subset=['start station name'])

In [35]:
station_mapping = df[['start station id', 'start station name']].drop_duplicates().set_index('start station id')

transitions_count = df.groupby(['start station id', 'end station id']).size().reset_index(name='count')

total_transitions = transitions_count.groupby('start station id')['count'].sum().reset_index(name='total')

transitions_prob = pd.merge(transitions_count, total_transitions, on='start station id')

transitions_prob['transition_prob'] = transitions_prob['count'] / transitions_prob['total']

station_probs = {}

for start_station_id in transitions_prob['start station id'].unique():
    start_station_name = station_mapping.loc[start_station_id, 'start station name']
    station_probs[start_station_name] = transitions_prob[transitions_prob['start station id'] == start_station_id].set_index('end station id')['transition_prob'].to_dict()

rows = []

for start_station_name, probs in station_probs.items():
    for end_station_id, prob in probs.items():
        rows.append((start_station_name, station_mapping.loc[end_station_id, 'start station name'], prob))

df_station_probs = pd.DataFrame(rows, columns=['Estação de Partida', 'Estação de Destino', 'Probabilidade'])

df_station_probs

,Estação de Partida,Estação de Destino,Probabilidade
0,W 52 St & 11 Ave,W 52 St & 11 Ave,0.020517
1,W 52 St & 11 Ave,Franklin St & W Broadway,0.008921
2,W 52 St & 11 Ave,Atlantic Ave & Fort Greene Pl,0.000892
3,W 52 St & 11 Ave,W 17 St & 8 Ave,0.004460
4,W 52 St & 11 Ave,Lexington Ave & Classon Ave,0.000892
...,...,...,...
68288,NYCBS Test,Lexington Ave & E 29 St,0.018349
68289,NYCBS Test,E 23 St & 1 Ave,0.009174
68290,NYCBS Test,1 Ave & E 18 St,0.009174
68291,NYCBS Test,E 60 St & York Ave,0.009174
